In [ ]:
import numpy as np
import slmsuite
from matplotlib import pyplot as plt
from slmsuite.holography.algorithms import Hologram,SpotHologram
from slmsuite.holography import analysis
from tools.IonChainTools import *
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import scipy.constants as con
from PIL import Image
from math import sin, pi as π
from slmsuite.holography.toolbox import _process_grid,pad
import cv2

In [ ]:
#loading the slm-200
from slmsuite.hardware.slms.santec import Santec
Santec.info(verbose=True)
#set wavelength to the correct value
slm = Santec(slm_number=1, display_number=2, wav_um=.730, settle_time_s=.3)

In [ ]:
# load the camera
from slmsuite.hardware.cameras.alliedvision import AlliedVision
AlliedVision.info(verbose=True)
#find the camera we want,then choose it
cam = AlliedVision(serial="0245P", verbose=True)
from slmsuite.hardware.cameraslms import FourierSLM
fs = FourierSLM(cam, slm)

In [ ]:
#load vendor-provided wavefront calibraiton file
slm.load_vendor_phase_correction(
 file_path='e:\A_my project in Seattle\project\A_intensity generator\slm_initiate\Wavefront_correction_Data_adjust(730nm).csv', 
 smooth=True
)

In [ ]:
#Fourier calibration
#Remember everytime we move the camera or change the setup we need to do the Fourier calibration again
#When I'm doing the fourier caliration, I put an OD2 and tune the current value to be 24mA for reference. For pratical use, if you find the calibration works not well, please close the camera with demand" cam.close" and then using the "vimba viewer" to find the best exposure time and current value so that the camera can detect the array dots

cam.set_exposure(100e-6) # set the exposure time for the camera

fs.fourier_calibrate(
    array_shape=[10, 10],           # Size of the calibration grid (Nx, Ny) [knm]
    array_pitch=[18, 18],           # Pitch of the calibration grid (x, y) [knm]
    plot=True
)

In [ ]:
#Remember to save the calibration data after calibration each time and next time you only need to load the data

# fs.save_fourier_calibration()
fs.load_fourier_calibration()

In [ ]:
# functions for caculating the ions' position on the camera
mCa40 = con.atomic_mass*39.9626

def ion_position_act(N,v): #N is the number of the ions, v is the trap frequency
    return calcPositions(N)*lengthScale(v)

def ion_position_cam(N,v,pixel_size,magnititude = 200):
    position = ion_position_act(N,v)*magnititude//pixel_size[0]
    return position

#Here we are using the MRAF algorithm to calculate the phase, the parameter rectan determines the signal region
#rectan =1: the signal region is a rectangle encloses all the ions; rectan = 0: the signal region is the circles around the ions
def excite_generator(ions,N,v,pixel_size,pixels,rectan = 1):
    position = np.array(ion_position_cam(N,v,pixel_size))
    dy = pixels[0]//8 
    all = range(N)
    pos = position[ions]
    position_cam = np.column_stack((pos+pixels[1]//2,np.full(pos.shape, pixels[0]//2+dy)))
    print(position_cam)
    position_cam = position_cam.T
    
    if rectan == 0:
        no_ions = np.array([ i for i in all if i not in ions])
        null = position[no_ions]
        null_points_cam = np.column_stack((null+pixels[1]//2,np.full(null.shape, pixels[0]//2+dy)))
        print(null_points_cam)
        null_points_cam = null_points_cam.T
        holo = SpotHologram(shape=(2048,2048), 
                spot_vectors=position_cam, 
                null_vectors=null_points_cam, 
                null_radius=100, 
                basis="ij", 
                cameraslm=fs)
    else:
        position_all = np.column_stack((position + pixels[1]//2, np.full(position.shape, pixels[0]//2 + dy)))
        mask_ij = np.zeros(pixels, dtype=float)
        pos1 = position_all[0]
        pos2 = position_all[-1]
        mask_ij[int(pos1[1])-150:int(pos1[1])+150, int(pos1[0])-150:int(pos2[0])+150] = 1.0
 
        def pad_to_shape(img, target_shape):
            pad_y = target_shape[0] - img.shape[0]
            pad_x = target_shape[1] - img.shape[1]
            return np.pad(img, ((0, pad_y), (0, pad_x)))

        mask_ij_padded = pad_to_shape(mask_ij, (4096, 4096))

        holo = SpotHologram(shape=(4096, 4096), 
                   spot_vectors=position_cam, 
                   basis="ij", 
                   cameraslm=fs,
                   null_region=mask_ij_padded)

    
    holo.optimize('WGS-Kim', feedback='computational_spot', stat_groups=['computational_spot'], maxiter=20)
    holo.optimize('WGS-Kim', feedback='experimental_spot', stat_groups=['computational_spot'], maxiter=10)
    
    phase = holo.extract_phase()
    
    return phase

#The function for writing the phase on the SLM and plot the image from the camera
def slm_write(ions,N,v,pixel_size,pixels,filter = 0,offset = 0,k=1,m=1,plot = 0,cal = 0):  
    phase  = excite_generator(ions,N,v,pixel_size,pixels,filter = 0,offset = 0,k=1)#m=4.71,c=0.7
    slm.write(phase)
    
    if plot == 1 or cal ==1:
        cam.set_exposure(100e-6)
        cam.flush()
        img = cam.get_image()
        position = ion_position_cam(N,v,pixel_size)
        dy = pixels[0]//8
        position_cam = np.column_stack((position+pixels[1]//2,np.full(position.shape, pixels[0]//2+dy)))
    
    if plot == 1:         #plot the image from the camera with circles enclosed the spots
        
        for i, pos in enumerate(position_cam):
        cv2.circle(img, (int(pos[0]), int(pos[1])), 60, (255, 0, 0), 2)
        plt.imshow(img)
        
    if cal ==1 :          #calculate the intensity on different ions
        inten = []
        inten.append(np.sum(img[1944//2-500-60:1944//2-500+60,2592//2-60:2592//2+60])/255.0)
        for i, pos in enumerate(position_cam):
            inten.append(np.sum(img[int(pos[1])-60:int(pos[1])+60,int(pos[0])-60:int(pos[0])+60])/255.0)
        return inten

In [ ]:
#the camera we are using has pixels:1944*2592, with pixel_size of 2.2um*2.2um

N = 6  # The number of the ions

v = 2*π*1e6   # trap frequency  
                 
pixel_size = np.array([2.2e-6,2.2e-6]) 
pixels = np.array([1944,2592])

ions = np.array([0,1,2,3,4,5]) #Here you determine the ions you want to excite.

slm_write(ions,N,v,pixel_size,pixels,filter = 0,offset = 0,k=1,m=1,plot = 0,cal = 0)

In [ ]:
#calculate the fidelity
result = slm_write(ions,N,v,pixel_size,pixels,filter = 0,offset = 0,k=1,m=1,plot = 0,cal = 1)
plt.scatter(range(7),result, c='blue', marker='o', label='data')
plt.xlabel('ions')
plt.ylabel('intensity')
# plt.ylim(1250, 2250)
# plt.yticks(np.arange(1250,2250,125))
plt.legend()
plt.show()

from numpy import sum, sqrt, max
import numpy as np
intensity = result[1:]

# background = result[0], Here is a little different from using the blaze grating phase, I recommend you use the background data calculated by using the blaze grating phase due to MRAF's property
# The below is the data in my test as a reference

back1 = [1359.75,1359.17,1358.62,1359.85,1359.08,1359.05,1359.29,1354.14,1357.32,1356.31,1356.35,1357.50,1356.73,1355.75,1356.11,1355.47,1356.42,1356.17,1355.81,1356.76]
back2 = [0.0980,0.0980,0.0980,0.0941,0.0941,0.0980,0.0980,0.0941,0.0941,0.0902,0.1020,0.0941,0.1059,0.1020,0.0980,0.0980,0.1059,0.1059,0.0941,0.0941,0.0941]
background = back1[0]

inten = np.array([ i- background for i in intensity])
inten[inten < 0] = 0
all = range(N)
undesire = [ i for i in all if i not in ions]
Omega_desire = sqrt(inten[ions])
Omega_undesire = sqrt(inten[undesire])
F = sum(Omega_desire)/(len(ions)*max(Omega_desire)) *(1-sum(Omega_undesire)/(len(undesire)*max(Omega_desire)))

print(F)